In [5]:
from datetime import datetime
from dateutil import relativedelta

now = datetime(year=2023, month=12, day=18, hour=12, minute=18, second=0)

print('현재시간:' + str(now))
print('------------월 연산-------------')
print(now + relativedelta.relativedelta(month=1))
print(now.replace(month=1))

현재시간:2023-12-18 12:18:00
------------월 연산-------------
2023-01-18 12:18:00
2023-01-18 12:18:00


In [73]:
import snowflake.connector
import pandas as pd

# 스노우플레이크 연결 설정
conn = snowflake.connector.connect(
    user='hdhsdwload',                # 스노우플레이크 사용자명
    password='Ghatyvld1!',            # 비밀번호
    account='dz48560.ap-northeast-2.aws',              # 계정 (예: xy12345.ap-southeast-2)
    warehouse='DW_LOAD_WH'          # 스키마 이름
)

try:
    # 커서 생성
    cursor = conn.cursor()

    # SQL 쿼리 실행
    query = "SELECT COUNT(*) FROM DW_LOAD_DB.HDHS_CU.CU_ITNT_CUST_GRD_1_INF"  # 조회할 테이블과 조건
    query2 = "alter stage DW_LOAD_DB.CONFIG.STG_AWS_S3 refresh"  # 조회할 테이블과 조건
    query3 = f"""
        SELECT 
            A.TABLE_SCHEMA,
            A.TABLE_NAME,
            LISTAGG('''' || REPLACE(A.RELATIVE_PATH, A.TABLE_SCHEMA || '/' || A.TABLE_NAME || '/' , '') || ''''  , ',')  WITHIN GROUP (ORDER BY A.LAST_MODIFIED)  AS FILE_NAME
        FROM 
            DW_LOAD_DB.CONFIG.V_S3_LIST A 
        WHERE A.LAST_MODIFIED BETWEEN TO_TIMESTAMP('20241223130000','YYYYMMDDHHMISS') 
                                  AND TO_TIMESTAMP('20241223150000','YYYYMMDDHHMISS')
          AND A.RELATIVE_PATH NOT LIKE '%LOAD%'
        GROUP BY ALL;
        """
    cursor.execute(query)
    print (cursor.fetchall())
    
    cursor.execute(query2)
    print (type(cursor.fetchone()[1]))
    
    colums = [row[0] for row in cursor.fetchall()]
    df = pd.read_sql(query3, conn)

    # 결과 출력

finally:
    # 연결 닫기
    cursor.close()
    conn.close()
    
df

[(24336442,)]


TypeError: 'NoneType' object is not subscriptable

In [50]:
df =pd.DataFrame(df)

In [51]:
df

,TABLE_SCHEMA,TABLE_NAME,FILE_NAME
0,HDHS_OD,OD_HPNT_PAY_APRVL_DTL_CRYPT,'2024/12/23/OD_HPNT_PAY_APRVL_DTL_CRYPT_202412...
1,HDHS_CU,CU_ARS_LDIN_MST_CRYPT,'2024/12/23/CU_ARS_LDIN_MST_CRYPT_20241223_030...
2,HDHS_OD,OD_CRD_APRVL_LOG_CRYPT,'2024/12/23/OD_CRD_APRVL_LOG_CRYPT_20241223_03...
3,HDHS_OD,OD_BASKT_INF,"'2024/12/23/20241223-040052103.csv','2024/12/2..."
4,HDHS_OD,OD_STLM_INF_CRYPT,'2024/12/23/OD_STLM_INF_CRYPT_20241223_030000....


In [53]:
df['TABLE_NAME'][0]

'OD_HPNT_PAY_APRVL_DTL_CRYPT'

In [126]:
# 스노우플레이크 연결 설정
conn = snowflake.connector.connect(
    user='hdhsdwload',                # 스노우플레이크 사용자명
    password='Ghatyvld1!',            # 비밀번호
    account='dz48560.ap-northeast-2.aws',              # 계정 (예: xy12345.ap-southeast-2)
    warehouse='DW_LOAD_WH'          # 스키마 이름
)

cursor = conn.cursor()

# for i,row in df.iterrows():
#     if df['TABLE_SCHEMA'][i] == "HDHS_OD" and df['TABLE_NAME'][i] != "OD_BASKT_INF":
#         
#         query5 = f"""
#         CALL DW_LOAD_DB.CONFIG.PROC_CDC_SUB('DW_LOAD_DB','HDHS_OD','{df['TABLE_NAME'][i]}','{df['FILE_NAME']}','20241223120000','20241223150000')
#         """
#         print(query5)
#         cursor.execute(query5)
#         print(cursor.fetchall())
        
query5 = f""" 
    CALL DW_LOAD_DB.CONFIG.PROC_CDC_SUB(
        'DW_LOAD_DB',
        'HDHS_OD',
        '{df['TABLE_NAME'][1]}',""" + str("''2024/12/23/CU_ARS_LDIN_MST_CRYPT_20241223_030000.csv','2024/12/23/20241223_040000.csv''") +""",
        '20241223120000',
        '20241223150000'
    )
    """
print(query5)
cursor.execute(query5)
print(cursor.fetchall())
        
        
cursor.close()
conn.close() 

[2024-12-23T16:42:05.870+0900] {connection.py:413} INFO - Snowflake Connector for Python Version: 3.12.2, Python Version: 3.11.10, Platform: Windows-10-10.0.22631-SP0
[2024-12-23T16:42:05.871+0900] {connection.py:1196} INFO - Connecting to GLOBAL Snowflake domain
[2024-12-23T16:42:05.872+0900] {connection.py:1277} INFO - This connection is in OCSP Fail Open Mode. TLS Certificates would be checked for validity and revocation status. Any other Certificate Revocation related exceptions or OCSP Responder failures would be disregarded in favor of connectivity.
 
    CALL DW_LOAD_DB.CONFIG.PROC_CDC_SUB(
        'DW_LOAD_DB',
        'HDHS_OD',
        'CU_ARS_LDIN_MST_CRYPT',''2024/12/23/CU_ARS_LDIN_MST_CRYPT_20241223_030000.csv','2024/12/23/20241223_040000.csv'',
        '20241223120000',
        '20241223150000'
    )
    


ProgrammingError: 001003 (42000): 01b937ce-0000-a9e7-0000-b5f5002540ee: SQL compilation error:
syntax error line 4 at position 34 unexpected '2024'.

In [103]:
fmrom airflow.providers.snowflake.operators.snowflake import SnowflakeOperator

CALL_PROCEDURE_QUERY = """
CALL DW_LOAD_DB.CONFIG.PROC_CDC_SUB(
    'DW_LOAD_DB',
    'HDHS_OD',
    '{{ params.table_name }}',
    '{{ params.file_name }}',
    '{{ params.start_time }}',
    '{{ params.end_time }}'
)
"""

call_procedure = SnowflakeOperator(
    task_id='call_snowflake_procedure',
    snowflake_conn_id='my_snowflake_conn',
    sql=CALL_PROCEDURE_QUERY,
    params={
        'table_name': {df['TABLE_NAME'][1]},
        'file_name': {df['FILE_NAME'][1]},
        'start_time': '20241223120000',
        'end_time': '20241223150000'
    },
)

print(call_procedure)


C:\Users\DKSYSTEMS\AppData\Local\Temp\ipykernel_23892\4178600287.py:14 AirflowProviderDeprecationWarning: Call to deprecated class SnowflakeOperator. (This class is deprecated. Please use `airflow.providers.common.sql.operators.sql.SQLExecuteQueryOperator`. Also, you can provide `hook_params={'warehouse': <warehouse>, 'database': <database>, 'role': <role>, 'schema': <schema>, 'authenticator': <authenticator>,'session_parameters': <session_parameters>}`.)

C:\ProgramData\anaconda3\envs\hd-ariflow\Lib\site-packages\airflow\models\param.py:75 RemovedInAirflow3Warning: The use of non-json-serializable params is deprecated and will be removed in a future release

<Task(SnowflakeOperator): call_snowflake_procedure>
